# CS418 Project1 - Exploratory Data Analysis

Find the Project Description [here](./Project_Description.pdf).

This project is done as part of **CS418 - Introduction to DataScience** at UIC.


In [307]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Load Dataset

In [308]:
demog = pd.read_csv('data/demographics_train.csv')
elect = pd.read_csv('data/election_train.csv')

In [309]:
print(elect.shape)
elect.head()

(2405, 6)


,Year,State,County,Office,Party,Votes
0,2018,AZ,Apache County,US Senator,Democratic,16298
1,2018,AZ,Apache County,US Senator,Republican,7810
2,2018,AZ,Cochise County,US Senator,Democratic,17383
3,2018,AZ,Cochise County,US Senator,Republican,26929
4,2018,AZ,Coconino County,US Senator,Democratic,34240


In [310]:
print(demog.shape)
demog.head()

(1216, 17)


,State,County,FIPS,Total Population,Citizen Voting-Age Population,"Percent White, not Hispanic or Latino","Percent Black, not Hispanic or Latino",Percent Hispanic or Latino,Percent Foreign Born,Percent Female,Percent Age 29 and Under,Percent Age 65 and Older,Median Household Income,Percent Unemployed,Percent Less than High School Degree,Percent Less than Bachelor's Degree,Percent Rural
0,Wisconsin,La Crosse,55063,117538,0,90.537528,1.214075,1.724549,2.976059,51.171536,43.241335,14.702479,51477,4.796952,5.474767,67.529757,16.827753
1,Virginia,Alleghany,51005,15919,12705,91.940449,5.207614,1.432251,1.300333,51.077329,31.660280,23.902255,45538,4.560986,15.537543,83.711604,52.393846
2,Indiana,Fountain,18045,16741,12750,95.705155,0.400215,2.359477,1.547100,49.770026,35.899887,18.941521,45924,7.978789,12.032155,85.538940,65.951276
3,Ohio,Geauga,39055,94020,0,95.837056,1.256116,1.294405,2.578175,50.678579,36.281642,18.028079,74165,4.036902,8.928599,62.730824,63.968990
4,Wisconsin,Jackson,55053,20566,15835,86.662453,1.983857,3.082758,1.376058,46.649810,36.292911,17.587280,49608,5.569698,11.792912,86.129256,72.238251


In [311]:
election_train = elect.pivot(index=['Year','State','County','Office'], columns='Party', values='Votes').reset_index()
print(election_train.shape)
election_train

(1205, 6)


Party,Year,State,County,Office,Democratic,Republican
0,2018,AZ,Apache County,US Senator,16298.0,7810.0
1,2018,AZ,Cochise County,US Senator,17383.0,26929.0
2,2018,AZ,Coconino County,US Senator,34240.0,19249.0
3,2018,AZ,Gila County,US Senator,7643.0,12180.0
4,2018,AZ,Graham County,US Senator,3368.0,6870.0
...,...,...,...,...,...,...
1200,2018,WY,Platte County,US Senator,801.0,2850.0
1201,2018,WY,Sublette County,US Senator,668.0,2653.0
1202,2018,WY,Sweetwater County,US Senator,3943.0,8577.0
1203,2018,WY,Uinta County,US Senator,1371.0,4713.0


In [312]:
state_abbr = {'AL': 'Alabama',
 'AK': 'Alaska',
 'AZ': 'Arizona',
 'AR': 'Arkansas',
 'CA': 'California',
 'CO': 'Colorado',
 'CT': 'Connecticut',
 'DE': 'Delaware',
 'FL': 'Florida',
 'GA': 'Georgia',
 'HI': 'Hawaii',
 'ID': 'Idaho',
 'IL': 'Illinois',
 'IN': 'Indiana',
 'IA': 'Iowa',
 'KS': 'Kansas',
 'KY': 'Kentucky',
 'LA': 'Louisiana',
 'ME': 'Maine',
 'MD': 'Maryland',
 'MA': 'Massachusetts',
 'MI': 'Michigan',
 'MN': 'Minnesota',
 'MS': 'Mississippi',
 'MO': 'Missouri',
 'MT': 'Montana',
 'NE': 'Nebraska',
 'NV': 'Nevada',
 'NH': 'New Hampshire',
 'NJ': 'New Jersey',
 'NM': 'New Mexico',
 'NY': 'New York',
 'NC': 'North Carolina',
 'ND': 'North Dakota',
 'MP': 'Northern Mariana Islands',
 'OH': 'Ohio',
 'OK': 'Oklahoma',
 'OR': 'Oregon',
 'PW': 'Palau',
 'PA': 'Pennsylvania',
 'RI': 'Rhode Island',
 'PR': 'Puerto Rico',
 'SC': 'South Carolina',
 'SD': 'South Dakota',
 'TN': 'Tennessee',
 'TX': 'Texas',
 'UT': 'Utah',
 'VT': 'Vermont',
 'VA': 'Virginia',
 'WA': 'Washington',
 'DC': 'Washington, DC',
 'WV': 'West Virginia',
 'WI': 'Wisconsin',
 'WY': 'Wyoming',
 'VI': 'Virgin Islands'}

In [313]:
election_train['State'] = election_train['State'].map(state_abbr)

In [314]:
def standardize_countyname(county):
    county = county.replace('County', '').strip()
    county = county.replace('City', '').strip()
    return county.lower()
    
election_train['County'] = election_train['County'].apply(standardize_countyname)
demog['County'] = demog['County'].apply(standardize_countyname)

In [316]:
election_train_merge = election_train.merge(demog, how='inner', on=['State', 'County'])
election_train_merge.shape

(1200, 21)

In [322]:
print('Shape: ', election_train_merge.shape)
election_train_merge.info()

Shape:  (1200, 21)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1200 entries, 0 to 1199
Data columns (total 21 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Year                                   1200 non-null   int64  
 1   State                                  1200 non-null   object 
 2   County                                 1200 non-null   object 
 3   Office                                 1200 non-null   object 
 4   Democratic                             1197 non-null   float64
 5   Republican                             1198 non-null   float64
 6   FIPS                                   1200 non-null   int64  
 7   Total Population                       1200 non-null   int64  
 8   Citizen Voting-Age Population          1200 non-null   int64  
 9   Percent White, not Hispanic or Latino  1200 non-null   float64
 10  Percent Black, not Hispanic or Latino  1200 non-null 

In [321]:
election_train_merge.describe()

,Year,Democratic,Republican,FIPS,Total Population,Citizen Voting-Age Population,"Percent White, not Hispanic or Latino","Percent Black, not Hispanic or Latino",Percent Hispanic or Latino,Percent Foreign Born,Percent Female,Percent Age 29 and Under,Percent Age 65 and Older,Median Household Income,Percent Unemployed,Percent Less than High School Degree,Percent Less than Bachelor's Degree,Percent Rural
count,1200.0,1197.000000,1198.000000,1200.000000,1.200000e+03,1.200000e+03,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000,1200.000000
mean,2018.0,25096.309106,20436.841402,38315.355000,1.208766e+05,3.226592e+04,79.099685,5.552660,10.556249,5.072053,49.825231,36.753853,18.105368,50098.516667,6.540929,13.448139,78.629091,55.950082
std,0.0,72593.640184,45218.050721,13001.996705,3.183773e+05,1.247969e+05,19.782542,9.274481,15.840002,6.054644,2.394356,5.618738,4.758807,12291.769508,2.776238,6.439020,9.191006,32.145514
min,2018.0,6.000000,46.000000,4001.000000,7.600000e+01,0.000000e+00,2.776702,0.000000,0.000000,0.000000,21.513413,11.842105,6.653188,21190.000000,0.000000,2.134454,26.335440,0.000000
25%,2018.0,1427.000000,2667.500000,27146.500000,1.208350e+04,0.000000e+00,70.168347,0.538615,1.819812,1.468692,49.352173,33.298306,15.052983,42081.500000,4.710440,8.916211,74.432551,29.744143
50%,2018.0,4213.000000,6691.000000,39140.000000,3.264300e+04,0.000000e+00,86.801005,1.605082,3.884901,2.869445,50.310516,36.427490,17.704921,48391.000000,6.417206,11.841157,81.013166,56.201291
75%,2018.0,14206.000000,16740.500000,48416.000000,8.582300e+04,1.893250e+04,93.876656,6.426589,11.075522,6.319101,50.971969,39.451774,20.501641,54958.250000,8.118306,16.811195,85.004467,83.377538
max,2018.0,881802.000000,672505.000000,56043.000000,4.434257e+06,2.723565e+06,99.627329,63.953279,95.479801,52.229868,56.418468,67.367823,37.622759,125672.000000,18.771186,49.673777,97.014925,100.000000
